# PassiveSSH Analysis Workshop

PassiveSSH is a powerful passive reconnaissance tool that collects and indexes SSH banners and host key fingerprints from internet-wide scans. This data can be invaluable for threat intelligence, security research, and infrastructure analysis.

## What is PassiveSSH?

PassiveSSH provides historical and current information about SSH services across the internet, including:
- SSH banner information (versions, implementations)
- Host key fingerprints and their associated hosts
- Historical data showing changes over time
- Correlation between different SSH characteristics

## Documentation

- Main Documentation: https://d4-project.github.io/passive-ssh/
- API Reference: Available through the web interface
- Data Sources: Internet-wide SSH scans and honeypot data

## Learning Objectives

By the end of this workshop, you will be able to:
1. Query SSH banner information and understand its significance
2. Identify hosts using specific SSH banners
3. Retrieve detailed SSH information for specific hosts
4. Track historical changes in SSH configurations
5. Correlate hosts using SSH fingerprints

## Exercises

### PassiveSSH API Setup

Before we begin exploring PassiveSSH data, we need to set up our authentication credentials. The PassiveSSH service requires user authentication to access the API endpoints.

**Prerequisites:**
- Valid PassiveSSH account credentials
- Network access to https://pssh.circl.lu

**Setup Instructions:**
1. Obtain your PassiveSSH username and API key from your account
2. Run the cell below to securely input your credentials
3. The session will be configured for all subsequent API calls

**Security Note:** Using `getpass` ensures your credentials are not displayed in the notebook output.

In [2]:
import getpass
import requests
import json

# Securely collect PassiveSSH credentials
# Note: These credentials are not stored in the notebook for security
print("Setting up PassiveSSH API authentication...")
PASSIVE_SSH_USER = getpass.getpass("Enter your PassiveSSH Username: ")
PASSIVE_SSH_KEY = getpass.getpass("Enter your PassiveSSH API Key: ")

# Create a persistent session with authentication
# This session will be reused for all API calls
pssh = requests.Session()
pssh.auth = (PASSIVE_SSH_USER, PASSIVE_SSH_KEY)

print("✓ Authentication configured successfully!")
print("Ready to query PassiveSSH API endpoints.")

Setting up PassiveSSH API authentication...


Enter your PassiveSSH Username:  ········
Enter your PassiveSSH API Key:  ········


✓ Authentication configured successfully!
Ready to query PassiveSSH API endpoints.


### Exercise 1.0: Retrieve All SSH Banners

**Objective:** Understand the scope and diversity of SSH implementations across the internet.

**What are SSH Banners?**
SSH banners are identification strings sent by SSH servers during the initial connection handshake. They typically contain:
- SSH protocol version (SSH-1.x or SSH-2.x)
- Software implementation name and version
- Sometimes additional identifying information

**Why This Matters:**
- Different SSH implementations have different security characteristics
- Banner analysis can help identify outdated or vulnerable SSH servers
- Unusual banners might indicate custom implementations or potential threats

**API Endpoint:** `https://pssh.circl.lu/banners`

**Expected Output:** 
- Total count of unique SSH banners in the database
- Sample of the most recently observed banners

**Performance Note:** This query retrieves a comprehensive dataset and may take some time to complete.

**Try It:** Run the cell below to see the diversity of SSH implementations on the internet.

In [2]:
# Query all SSH banners from PassiveSSH
print("Retrieving all SSH banners from PassiveSSH database...")
print("This may take a moment as it fetches a large dataset...")

try:
    response = pssh.get('https://pssh.circl.lu/banners')
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()
    
    print(f"Successfully retrieved banner data!")
    print(f"Total unique SSH banners in database: {len(data['banners'])}")
    
    # Show the last 10 banners (most recently added/observed)
    banner_list = list(data['banners'].keys())
    bottom10_banners = banner_list[-10:]
    
    print(f"\nShowing the 10 most recently observed banners:")
    for i, banner in enumerate(bottom10_banners, 1):
        host_count = data['banners'][banner]
        print(f"{i:2d}. {banner} ({host_count} hosts)")
    
    print(f"\nTry examining some of these banners in the next exercise!")
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching banner data: {e}")
except KeyError as e:
    print(f"Unexpected response format: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Retrieving all SSH banners from PassiveSSH database...
This may take a moment as it fetches a large dataset...
Successfully retrieved banner data!
Total unique SSH banners in database: 185196

Showing the 10 most recently observed banners:
 1. SSH-2.0-cdFhHtuR3Ixb5I (1 hosts)
 2. SSH-2.0-WsK-B (1 hosts)
 3. SSH-2.0-PwNQf (1 hosts)
 4. SSH-2.0-Rp1a1YLz-cEhqy_ (1 hosts)
 5. SSH-2.0-vg9PhdmMYh (1 hosts)
 6. SSH-2.0-DYeKzgZ5_icLCpJ (1 hosts)
 7. SSH-2.0-jau-q (1 hosts)
 8. SSH-2.0-OKHfk4qa4rY_ (1 hosts)
 9. SSH-2.0-9M4hJgjfT9BFs (1 hosts)
10. SSH-2.0-ZZHG86XAPc (1 hosts)

Try examining some of these banners in the next exercise!


### Exercise 1.1: Find Hosts Using a Specific SSH Banner

**Objective:** Identify all hosts that present a particular SSH banner string.

**Use Cases:**
- **Vulnerability Research:** Find all hosts running a specific vulnerable SSH version
- **Infrastructure Mapping:** Identify hosts belonging to the same organization or using the same configuration
- **Threat Hunting:** Locate hosts with suspicious or unusual SSH banners
- **Compliance Monitoring:** Track deployment of specific SSH implementations

**About This Banner:**
The banner `SSH-2.0-GIfQrwMLPdNQT99` appears to be a non-standard or potentially custom SSH implementation. Standard SSH banners typically follow patterns like:
- `SSH-2.0-OpenSSH_8.2` (OpenSSH)
- `SSH-2.0-libssh_0.8.8` (libssh)
- `SSH-2.0-Cisco-1.25` (Cisco devices)

**API Endpoint:** `https://pssh.circl.lu/banner/hosts/[banner]`

**Expected Output:**
- List of IP addresses using this specific banner
- Timestamps of when the banner was observed
- Additional metadata about each host

**Analysis Questions:**
- How many hosts use this banner?
- Are the hosts geographically clustered?
- When was this banner first/last observed?

In [5]:
# Query hosts using a specific SSH banner
banner = 'SSH-2.0-GIfQrwMLPdNQT99'
print(f"Searching for hosts using SSH banner: '{banner}'")

try:
    response = pssh.get(f"https://pssh.circl.lu/banner/hosts/{banner}")
    response.raise_for_status()
    data = response.json()
    
    print(f"Query successful!")
    
    # Analyze the results
    if 'hosts' in data and data['hosts']:
        host_count = len(data['hosts'])
        print(f"Found {host_count} host(s) using this banner")
        
        print(f"\nDetailed results:")
        print(json.dumps(data, indent=2))
        
        # Additional analysis
        print(f"\nAnalysis:")
        print(f"   • This banner appears to be non-standard")
        print(f"   • Standard banners usually follow 'SSH-2.0-[Software]_[Version]' format")
        print(f"   • Custom banners might indicate specialized implementations")
        
    else:
        print("No hosts found using this specific banner")
        
except requests.exceptions.RequestException as e:
    print(f"Error querying banner hosts: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Searching for hosts using SSH banner: 'SSH-2.0-GIfQrwMLPdNQT99'
Query successful!
Found 1 host(s) using this banner

Detailed results:
{
  "banner": "SSH-2.0-GIfQrwMLPdNQT99",
  "hosts": [
    "216.75.130.11"
  ]
}

Analysis:
   • This banner appears to be non-standard
   • Standard banners usually follow 'SSH-2.0-[Software]_[Version]' format
   • Custom banners might indicate specialized implementations


### Exercise 1.2: Retrieve Complete SSH Information for a Host

**Objective:** Gather comprehensive SSH service information for a specific IP address.

**What Information Do We Get?**
- **Current SSH Banner:** The most recently observed banner string
- **Host Key Fingerprints:** Unique identifiers for the server's SSH keys
- **Key Types:** RSA, DSA, ECDSA, or Ed25519 keys
- **First/Last Seen:** Temporal information about SSH service observation
- **Port Information:** Which ports the SSH service was observed on

**About This Host (216.75.130.11):**
This IP address was selected from the previous query results. By analyzing this specific host, we can understand:
- What SSH implementation it's running
- How long it has been observed
- Whether its SSH configuration has changed over time

**Security Implications:**
- **Key Fingerprints:** Help verify server authenticity and detect MITM attacks
- **Banner Analysis:** Reveals software versions and potential vulnerabilities
- **Temporal Data:** Shows infrastructure stability and configuration changes

**API Endpoint:** `https://pssh.circl.lu/host/ssh/[host]`

**Expected Output:**
- Detailed SSH service configuration
- Cryptographic key information
- Observation timestamps and metadata

**Analysis Questions:**
- What SSH software is this host running?
- How many different key types does it present?
- Is this a long-running or recently deployed service?

In [6]:
# Query comprehensive SSH information for a specific host
host = '216.75.130.11'
print(f"Retrieving SSH information for host: {host}")

try:
    response = pssh.get(f"https://pssh.circl.lu/host/ssh/{host}")
    response.raise_for_status()
    data = response.json()
    
    print(f"Successfully retrieved SSH data for {host}")
    
    # Display formatted results
    print(f"\nComplete SSH Information:")
    print(json.dumps(data, indent=2))
    
    # Perform basic analysis if data is available
    if data:
        print(f"\nQuick Analysis:")
        
        # Check for banner information
        if 'banner' in data:
            print(f"   • SSH Banner: {data['banner']}")
            
        # Check for fingerprints
        if 'fingerprints' in data:
            fp_count = len(data['fingerprints'])
            print(f"   • Host Key Fingerprints: {fp_count} found")
            
        # Check for timestamps
        if 'first_seen' in data:
            print(f"   • First Observed: {data['first_seen']}")
        if 'last_seen' in data:
            print(f"   • Last Observed: {data['last_seen']}")
            
        print(f"   • This data helps verify server identity and track changes")
    
except requests.exceptions.RequestException as e:
    print(f"Error querying host SSH info: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Retrieving SSH information for host: 216.75.130.11
Successfully retrieved SSH data for 216.75.130.11

Complete SSH Information:
{
  "first_seen": "20210203",
  "last_seen": "20210203",
  "ports": [
    "22"
  ],
  "banner": [
    "SSH-2.0-GIfQrwMLPdNQT99"
  ],
  "hassh": {
    "513c16a003f0c2b69f5968e462635b90": [
      {
        "key": [
          "ssh-rsa",
          "ssh-ed25519"
        ],
        "encrypt": [
          "chacha20-poly1305@openssh.com",
          "aes128-ctr",
          "aes192-ctr",
          "aes256-ctr"
        ],
        "mac": [
          "umac-64-etm@openssh.com",
          "umac-128-etm@openssh.com",
          "hmac-sha2-256-etm@openssh.com",
          "hmac-sha2-512-etm@openssh.com",
          "hmac-sha1-etm@openssh.com",
          "umac-64@openssh.com",
          "umac-128@openssh.com",
          "hmac-sha2-256",
          "hmac-sha2-512",
          "hmac-sha1",
          "hmac-ripemd160-etm@openssh.com",
          "hmac-ripemd160",
          "hmac-ripemd16

### Exercise 1.3: Analyze Historical Changes for a Host

**Objective:** Understand how a host's SSH configuration has evolved over time.

**Why Historical Data Matters:**
- **Infrastructure Changes:** Track server updates, migrations, or reconfigurations
- **Security Evolution:** Monitor adoption of stronger SSH implementations
- **Incident Investigation:** Correlate SSH changes with security events
- **Compliance Tracking:** Verify that security updates were applied
- **Threat Intelligence:** Identify patterns in attacker infrastructure

**Types of Changes to Look For:**
- **Banner Changes:** Software updates or complete system changes
- **Key Rotations:** New host keys being deployed
- **Port Changes:** SSH service moving to different ports
- **Service Interruptions:** Periods when SSH was unavailable

**Forensic Value:**
Historical SSH data can provide crucial timeline information for:
- When a compromised server was first observed
- Whether attackers changed SSH configurations
- How infrastructure evolved during an incident

**API Endpoint:** `https://pssh.circl.lu/host/history/[host]`

**Expected Output:**
- Chronological list of SSH configuration changes
- Timestamps for each observed state
- Details about what changed between observations

**Analysis Questions:**
- How often does this host change its SSH configuration?
- Are there any suspicious or unusual changes?
- Does the change pattern suggest automated updates or manual intervention?

In [10]:
# Query historical SSH configuration changes for a host
host = '216.75.130.11'
print(f"Retrieving SSH history for host: {host}")

try:
    response = pssh.get(f"https://pssh.circl.lu/host/history/{host}")
    response.raise_for_status()
    data = response.json()
    
    print(f"Successfully retrieved historical data for {host}")
    
    # Display formatted results
    print(f"\n SSH Configuration History:")
    print(json.dumps(data, indent=2))
    
    # Analyze the historical data
    if data:
        print(f"\n Timeline Analysis:")
        print(f"   • Total historical records: {len(data["history"])}")
        
        # Extract timestamps if available
        timestamps = []
        banners = set()
        
        for timestamp in list(data["history"].keys()):
            timestamps.append(timestamp)
            for banner in data["history"][timestamp]:
                banners.add(banner)
        
        if timestamps:
            print(f"   • Observation period: {min(timestamps)} to {max(timestamps)}")
        
        if banners:
            print(f"   • Unique SSH banners observed: {len(banners)}")
            for banner in banners:
                print(f"     - {banner}")
        
        print(f"   • Changes in SSH config can indicate updates, migrations, or incidents")
    
except requests.exceptions.RequestException as e:
    print(f"Error querying host history: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Retrieving SSH history for host: 216.75.130.11
Successfully retrieved historical data for 216.75.130.11

 SSH Configuration History:
{
  "hosts": "216.75.130.11",
  "history": {
    "1612315016": [
      "ssh-ed25519;f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a",
      "ssh-rsa;9a:ea:5b:86:51:70:34:25:0c:e6:ec:99:89:8f:6e:36"
    ]
  }
}

 Timeline Analysis:
   • Total historical records: 1
   • Observation period: 1612315016 to 1612315016
   • Unique SSH banners observed: 2
     - ssh-rsa;9a:ea:5b:86:51:70:34:25:0c:e6:ec:99:89:8f:6e:36
     - ssh-ed25519;f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a
   • Changes in SSH config can indicate updates, migrations, or incidents


### Exercise 1.4: Discover Hosts Sharing SSH Key Fingerprints

**Objective:** Identify all hosts that share the same SSH host key fingerprint.

**What are SSH Host Key Fingerprints?**
SSH host key fingerprints are cryptographic hashes of a server's public keys. They serve as unique identifiers and are used for:
- **Server Authentication:** Clients verify they're connecting to the correct server
- **Man-in-the-Middle Detection:** Changed fingerprints indicate potential attacks
- **Infrastructure Correlation:** Multiple hosts sharing keys suggests common management

**Why Multiple Hosts Share Fingerprints:**
- **Load Balancers:** Multiple servers behind a load balancer may share keys
- **Virtual Machine Cloning:** VMs cloned from the same template inherit SSH keys
- **Configuration Management:** Automated deployment tools may distribute the same keys
- **Compromised Infrastructure:** Attackers might reuse SSH keys across multiple systems

**Security Implications:**
- **Legitimate Sharing:** Common in properly managed infrastructure
- **Security Risk:** Shared keys increase the impact of key compromise
- **Threat Intelligence:** Unusual sharing patterns may indicate malicious infrastructure

**About This Fingerprint:**
The fingerprint `f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a` represents an MD5 hash of an SSH host key. We'll discover how many hosts present this same key.

**API Endpoint:** `https://pssh.circl.lu/fingerprint/all/[fingerprint]`

**Expected Output:**
- List of all IP addresses sharing this fingerprint
- Temporal information about when each host was observed
- Additional context about each host

**Analysis Questions:**
- How many hosts share this fingerprint?
- Are the hosts geographically or network-wise related?
- Could this represent legitimate infrastructure or something suspicious?
- What can we infer about the organization or entity operating these hosts?

In [8]:
fingerprint = 'f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a'
data = pssh.get(f"https://pssh.circl.lu/fingerprint/all/{fingerprint}").json()
print(json.dumps(data, indent=4))

{
    "type": "ssh-ed25519",
    "first_seen": "20210203",
    "last_seen": "20210203",
    "base64": "ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGHTXANkr7LIygUuVvp8h7Lx6w3b1747ZQ1CjrS5+rhI",
    "crypto_material": {
        "len_pk": "32",
        "pk": "44247786095392330785659151135084085251596279940380466642510923029929107568712"
    },
    "fingerprint": "f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a",
    "hosts": [
        "216.75.130.11"
    ]
}


### Task - Pivot from banners to hosts and fingerprints

1. Pick a banner from the output of Exercise 1.0 (for example `SSH-2.0-GIfQrwMLPdNQT99`).
2. Query the hosts for that banner (`/banner/hosts/[banner]`) and collect up to 5 IPs.
3. For each selected IP:
    - Fetch full SSH info (`/host/ssh/[ip]`) and record: banner, host key fingerprints, key types, first_seen, last_seen.
    - Fetch historical data (`/host/history/[ip]`) and record number of historical records and any banner changes.
4. For every fingerprint you find, query `/fingerprint/all/[fingerprint]` to see if the same key appears on other hosts.
5. Produce a short report (3–5 bullets) summarizing:
    - How many hosts used the banner and how many you inspected
    - Any unusual banners or multiple key types found
    - Evidence of shared fingerprints (and possible explanations: load balancer, VM cloning, or misconfiguration)

Hints:
- Reuse the `pssh` session already created in the notebook.
- Use `.json()` on responses and handle HTTP/errors gracefully.
- Limit results to a few hosts to keep queries fast.


In [ ]:

# Your code here...
